# Health Innovation Analysis Using MeSH Terms as Document Labels

In [1]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:
import os
import ast
import json
import itertools

import pandas as pd
import numpy as np

from collections import defaultdict, Counter
from datetime import datetime
from itertools import zip_longest

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
%matplotlib inline

#NB Open a standard set of directories
sep = os.sep
#Paths
#Get the top path
top_path = os.path.dirname(os.getcwd())
#Create the path for external data
ext_data = os.path.join(top_path,'data{s}external{s}'.format(s=sep))
#Raw path (for html downloads)
raw_data = os.path.join(top_path,'data{s}raw{s}'.format(s=sep))
#And external data
proc_data = os.path.join(top_path,'data{s}processed{s}'.format(s=sep))
#And path for figures
fig_path = os.path.join(top_path,'reports{s}figures{s}'.format(s=sep))

#Get date for saving files
today = datetime.utcnow()
today_str = "_".join([str(x) for x in [today.month, today.day, today.year]])

In [5]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [6]:
def column_to_list(df, c):
    return [ast.literal_eval(i) for i in df[c]]

In [7]:
def grouper(n, iterable, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

## Labelling Using the NLM Batch MTI Tool

As part of their Indexing Initiative, NIH and NLM have developed an algorithm that will suggest MeSh terms as labels for a document. A public tool that makes use of their technology is the [MeSH on Demand](https://meshb.nlm.nih.gov/MeSHonDemand), which can process a single text at a time. However, with an account, users can access a series of [batch processing tools](https://ii.nlm.nih.gov/Batch/index.shtml), which can perform the same kind of labelling on a corpus of texts, along with a greater degree of control over parameters such as term filtering and outputs.

The tools require uploaded texts to be formated in one of a set of specific styles, which can be found [here](https://ii.nlm.nih.gov/Help/index.shtml). Here we import the descriptions that we want to be labelled from the _Grant Database_, and write them to a text file in the "Single Line Delimited Input w/ ID" format, with ASCII characters only.

In [8]:
gdb_df = pd.read_csv(raw_data + 'health_cb_gh_gdb.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

/Users/grichardson/miniconda3/envs/rwjf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,7,9,12,13,14,15,16,20,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
descriptions = gdb_df['Description']

index_pad_length = len(str(max(descriptions.index)))

with open(proc_data + 'gdb_descriptions_for_mti.txt', 'w') as f:
    for i, t in zip(descriptions.index, descriptions.values):
        if pd.isnull(t):
            t = 'None'
        f.write('{}|{}\n'.format(str(i).rjust(index_pad_length, '0'),
                                 t.replace("\n", " ").replace("\r", " ").replace("\t", " ").encode('ascii', 'ignore')))

This text file was uploaded to the [Generic Batch with Validation](https://ii.nlm.nih.gov/Batch/UTS_Required/genericWithValidation.shtml) tool using "Single Line Delimited Input w/ ID" as the only _Batch Specific_ option, and the execution arguments (recommended by an NLM employee):

```
MTI.Linux -MoD_PP -trackPositional -E
```

Upon completion of the job (which was run overnight), a link to the results are emailed to the user. From here we can download the file `text.out`, which contains the results for every document in the text uploaded. For each document, the results take the following form:

```
000000|PRC|17768168;17211657;24593272;25377753;22960242;17691196;8705186;20643683;25030295;9682210|
000000|Protons|D011522|C0033727|52674|1195^7^0
000000|Radioactive Waste|D011850|C0034552|46935|1413^17^0
000000|Universities|D014495|C0041740|27039|1830^12^0
000000|Diamond|D018130|C0057717|9522|1022^7^0
000000|Nuclear Energy|D009678|C0028572|29955|1354^14^0
000000|Neutrons|D009502|C0027946|22098|953^7^0
000000|Ions|D007477|C0022023|20340|1213^4^0
000000|Physics|D010825|C0031837|7722|531^7^0;251^7^0
000000|Students|D013334|C0038492|2466|1675^17^0;1684^8^0
000000|Neoplasms|D009369|C0027651|999|1227^6^0
000000|United Kingdom|D006113|C0041700|1000|86^6^0
000000|Cell Proliferation|D049109|C0596290|1000|1321^13^0
```

The number on the left is the ID we assigned to each document in the input file, and the first line of each document's results contains the PubMed unique IDs for the top 10 articles which are similar to the one uploaded. More information on interpreting outputs from the MTI tools can be found [here](https://ii.nlm.nih.gov/resource/MTI_output_help_info.html). The following rows contain the MeSH terms that were matched from the text. These are found using a combination of keyword matching, fuzzy matching, and similarity of the document to other documents in the PubMed archive. Each row contains the MeSH term, its CUI, a score, and the position of the word of phrase in the original document that gave rise to the label. This positional information consists of three components start position (zero offset), length, and a flag if the text is split (e.g. "community care" in "community of care"). Behind the scenes, the text input is reformatted, and as a result, the start and end indicies are off by 13 plus the length of your input index values.

For reference, here is line from the original text input file that gave the results above:

_000000|The past decade has seen a renaissance in accelerator R&amp;D in the UK, building upon the existing expertise at the Rutherford Appleton and Daresbury Laboratories and fuelled by the need to prepare for the generation of particle physics facilities after the LHC. Both the University of Oxford and Royal Holloway University of London have made very significant contributions to this renewed programme. In 2004, the John Adams Institute for Accelerator Science was created, jointly hosted by the Departments of Physics of the University of Oxford and Royal Holloway University of London, with support from PPARC and CCLRC, now merged into STFC. The initial programme was focussed on R&amp;D for the Linear Collider and the Neutrino Factory, but has broadened considerably since its inception to include developments of advanced and novel light sources, work on the upgrades for the Large Hadron Collider at CERN, the ISIS spallation neutron source at the Rutherford Appleton Laboratory, and of the new Diamond Light Source on the Harwell Science and Innovation Campus, and the development of novel accelerators for a variety of applications from medicine (for example, using protons and light ions to treat cancer therapy) to energy (where accelerator-driven sub-critical reactors could contribute to proliferation-safe generation of nuclear energy and help reduce the volume of highly active radioactive waste). A key part of the strategy is the training of a new generation of accelerator scientists able to design, build and operate the new facilities that would be required in the future and we have made excellent progress on this, with more than 20 graduate students and 15 PDRAs being trained by the JAI alone. We propose to continue this programme, as part of a broadly-based collaboration between the universities, the accelerator science institutes (John Adams Institute, the Cockcroft Institute and the STFC Accelerator Science and Technology Centre), and the national laboratories. A key objective is to encourage the development of a domestic industry able to support this work._

## Importing Batch Processed MeSH Labels

After downloading and renameing our `text.out` results file, we can import it and match it up to the original dataset. The following function is slightly lengthy, but it essentially parses the results and aggregates all of the labels associated with each document. It also sorts them in the original order that they were found in the document.

In [10]:
with open(raw_data + 'gdb_mesh_on_demand_labels.out', 'r') as f:
    mesh_on_demand_labels = f.read().split('\n')[:-1]

In [11]:
def parse_mti_batch_output(batch_output, descriptions, log_every=10000):
    mti_output_dict = defaultdict(dict)
    mti_output_dict['pub_med_uids'] = defaultdict(list)
    mti_output_dict['mesh_duis'] = defaultdict(list)
    mti_output_dict['mesh_cuis'] = defaultdict(list)
    mti_output_dict['mesh_labels'] = defaultdict(list)
    mti_output_dict['scores'] = defaultdict(list)
    mti_output_dict['original_phrases'] = defaultdict(list)
    mti_output_dict['term_counts'] = defaultdict(list)
    mti_output_dict['mesh_term_token_start_idx'] = defaultdict(list)
    mti_output_dict['mesh_term_token_end_idx'] = defaultdict(list)
    
    doc_vocab_to_mesh_map = defaultdict(list)
    
    batch_output_list = batch_output.copy()
    
    # loop through documents and get only the mesh label outputs for that doc
    for doc_id in range(len(descriptions)):
        
        doc_results = []
        
        original_phrases = []
        mesh_labels = []
        cui_codes = []
        starts = []
        ends = []
        term_counts = []
        
        if not doc_id % log_every:
            print("Processed:", doc_id)

        for row_id, result_row in enumerate(batch_output_list):
            row_doc_id = int(result_row.split('|')[0]) 
            if row_doc_id == doc_id:
                doc_results.append(result_row)
            else:
                break
            cutoff = row_id + 1
        # drop the labels for the current document from the original labels list
        # about 60x faster than re-indexing
        del batch_output_list[:cutoff]
        
        # cycle through labels in doc and add them to dicts
        for doc_row in doc_results:
            elements = doc_row.split('|')
            
            i = int(elements[0])
            
            # deal with the first line that only contains PubMed IDs for the doc
            if elements[1] == 'PRC':
                pub_med_uids = elements[-2].split(';')
                mti_output_dict['mesh_term_token_idx'][i] = pub_med_uids
            # deal with the actual MeSH labels
            else:
                ml = elements[1]
                mti_output_dict['mesh_labels'][i].append(ml)
                mti_output_dict['mesh_duis'][i].append(elements[2])
                mti_output_dict['mesh_cuis'][i].append(elements[3])
                mti_output_dict['scores'][i].append(elements[4])
            
                positions = elements[-1].split('^')

                if len(positions) > 3:
                    positions = flatten([p.split(';') for p in positions])
                positions = list(grouper(3, positions))

#                 starts.append([int(p[0]) for p in positions])
                term_counts.append(len(positions))
                mesh_labels.append([ml])
                
                original_phrases_row = []
                for position in positions:
                    if isinstance(position, str):
                        continue
                    elif None in position:
                        continue
                    else:
                        start = int(position[0]) - 15 - len(elements[0])
                        starts.append(start)
                        end = start + int(position[1])
                        ends.append(end)
                        original_phrases_row.append(descriptions[i][start:end])
                
                # map phrases from text to their MeSH labels in separate dict
                for opr in original_phrases_row:
                    original_phrases.append(opr)
                    if ml not in doc_vocab_to_mesh_map[opr]:
                        doc_vocab_to_mesh_map[opr].append(ml)

        mti_output_dict['original_phrases'][i] = original_phrases
        
        # get the phrases from the texts in their original order
#         starts = flatten(starts)
        original_phrases_ordered = [op for (s, op) in sorted(zip(starts, original_phrases), key=lambda pair: pair[0])]
        mti_output_dict['original_phrases'][i] = original_phrases_ordered
        # and the mesh labels
        mesh_labels_ordered = flatten([[ml] * tc for ml, tc in zip(mesh_labels, term_counts)])
        mesh_labels_ordered = flatten([ml for (s, ml) in sorted(zip(starts, mesh_labels_ordered), key=lambda pair: pair[0])])
        mti_output_dict['mesh_labels'][i] = mesh_labels_ordered
        
        cui_ordered = flatten([[cui] * tc for cui, tc in zip(mti_output_dict['mesh_cuis'][i], term_counts)])
        cui_ordered = [cui for (s, cui) in sorted(zip(starts, cui_ordered), key=lambda pair: pair[0])]
        mti_output_dict['mesh_cuis'][i] = cui_ordered
        
        dui_ordered = flatten([[dui] * tc for dui, tc in zip(mti_output_dict['mesh_duis'][i], term_counts)])
        dui_ordered = [dui for (s, dui) in sorted(zip(starts, dui_ordered), key=lambda pair: pair[0])]
        mti_output_dict['mesh_duis'][i] = dui_ordered
        
        mti_output_dict['mesh_term_token_start_idx'][i] = sorted(starts)
        mti_output_dict['mesh_term_token_end_idx'][i] = sorted(ends)
        mti_output_dict['term_counts'][i] = term_counts

    return mti_output_dict, doc_vocab_to_mesh_map

In [12]:
# takes a little while to run
mti_output_dict, vocab_mesh_mapping = parse_mti_batch_output(mesh_on_demand_labels, descriptions)

Processed: 0
Processed: 10000
Processed: 20000
Processed: 30000
Processed: 40000
Processed: 50000
Processed: 60000
Processed: 70000
Processed: 80000
Processed: 90000
Processed: 100000
Processed: 110000
Processed: 120000
Processed: 130000
Processed: 140000


The final step here is to merge the data with the original GDB and export it.

In [13]:
gdb_df['doc_id'] = gdb_df.index.values
for c in ['original_phrases', 'mesh_labels', 'pub_med_uids', 'mesh_labels', 'mesh_cuis', 'mesh_duis', 'mesh_term_token_start_idx', 'mesh_term_token_end_idx']:
    gdb_df[c] = gdb_df['doc_id'].map(mti_output_dict[c])

In [17]:
gdb_df.to_csv(proc_data + 'gdb_{}.csv'.format(today_str), index=False)

## Exploring the Labels

Let's have a look at some features of the labels look like for the processed documents.

In [23]:
gdb_df = pd.read_csv(proc_data + 'gdb_5_25_2018.csv')

/Users/grichardson/miniconda3/envs/rwjf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,5,7,10,11,12,13,14,18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
mesh_labels = column_to_list(gdb_df, 'mesh_labels')
mesh_label_counts = Counter(flatten(mesh_labels))

original_phrases = column_to_list(gdb_df, 'original_phrases')
original_phrases_counts = Counter(flatten(original_phrases))

Humans    	93553	students	34978
Students  	69067	research	19420
Animals   	37857	goal	16382
Research  	28895	patients	13999
Goals     	27021	human	13469
Universities	25024	brain	13109
Research Personnel	20361	the UK	12073
Female    	18103	University	11647
United States	17793	children	11019
Brain     	17582	HIV	10394
United Kingdom	15941	public health	8765
HIV Infections	14882	water	8471
Public Health	12587	software	7347
Child     	12574	goals	6961
Europe    	11503	award	6917
Software  	11452	Europe	6763
Neoplasms 	10890	investigators	6612
Surveys and Questionnaires	10299	student	6544
Awards and Prizes	9880	faculty	6488
Fellowships and Scholarships	9163	individuals	6398


### Label and Term Frequencies

In [41]:
print('Most common MeSH Labels and original terms:\n')
print('{}\t{:30}\t{}\t{:20}\t{}\n'.format('Rank', 'MeSH Label', 'Count', 'Original Term', 'Count'))

for i, (ml, op) in enumerate(zip(mesh_label_counts.most_common(20), original_phrases_counts.most_common(20))):
    print('{}\t{:30}\t{}\t{:20}\t{}'.format(i, ml[0], ml[1], op[0], op[1]))

print('\nNumber of unique MeSH Labels:', len(set(flatten(mesh_labels))))
print('Number of unique original terms:', len(set(flatten(original_phrases))))

Most common MeSH Labels and original terms:

Rank	MeSH Label                    	Count	Original Phrase     	Count

0	Humans                        	93553	students            	34978
1	Students                      	69067	research            	19420
2	Animals                       	37857	goal                	16382
3	Research                      	28895	patients            	13999
4	Goals                         	27021	human               	13469
5	Universities                  	25024	brain               	13109
6	Research Personnel            	20361	the UK              	12073
7	Female                        	18103	University          	11647
8	United States                 	17793	children            	11019
9	Brain                         	17582	HIV                 	10394
10	United Kingdom                	15941	public health       	8765
11	HIV Infections                	14882	water               	8471
12	Public Health                 	12587	software            	7347
13	Child                   

The most common MeSH label is _Humans_, however this is not the case for the original document terms. This is because the MeSH on Demand labelling parameters works on inference as well as exact matching. There can also be multiple MeSH labels for each term identified in the document. Overall however, there are a smaller set of MeSH labels compared to the original terms that generated them. There are a lot of shared terms, revealing some of the largest topics and actors to be covered in the dataset.

In [43]:
n = 5
mesh_label_counts_low = Counter({k: v for k, v in mesh_label_counts.items() if v < n})
original_phrases_counts_low = Counter({k: v for k, v in original_phrases_counts.items() if v < n})

In [48]:
print('Most common MeSH Labels and original terms:\n')
print('{}\t{:40}\t{}\t{:40}\t{}\n'.format('Rank', 'MeSH Label', 'Count', 'Original Term', 'Count'))

for i, (ml, op) in enumerate(zip(mesh_label_counts_low.most_common(30), original_phrases_counts_low.most_common(30))):
    print('{}\t{:40}\t{}\t{:40}\t{}'.format(i, ml[0], ml[1], op[0], op[1]))

Most common MeSH Labels and original terms:

Rank	MeSH Label                              	Count	Original Term                           	Count

0	CDC20 protein, human                    	4	muscarinic                              	4
1	Cdc20 Proteins                          	4	sinsemilla                              	4
2	Hepatorenal Syndrome                    	4	n Cypr                                  	4
3	visilizumab                             	4	judgements                              	4
4	Cyclic Nucleotide-Gated Cation Channels 	4	cell membrane proteins                  	4
5	cocoa butter                            	4	endosome membrane                       	4
6	Burkholderia cepacia                    	4	cellular ageing                         	4
7	Immunoglobulin Light-chain Amyloidosis  	4	viscera                                 	4
8	Retreatment                             	4	nHH                                     	4
9	Kidney Tubules, Collecting              	4	idiopathic hypogon

Looking at some of the labels and terms with low counts, we can see that there is much more variety, and some very specific concepts have been captured, from chemicals, to ailments, to body parts and some treatments.

### Matching MeSH Labels to Groups

A feature of the MeSH terms is that they are organised in a tree structure, with each term being nested underneath a larger grouping. Some of the tree branches are up to 13 nodes deep. We can take the MeSH ontology (pre-processed from the available XML file), and check how this matches up to the labelled terms in the documents. We do this using the Descriptor Unique ID.

In [24]:
with open(proc_data + 'mesh_codes_processed_DUI_5_21_2018.json', 'r') as f:
    mesh_codes = json.load(f)

In [56]:
print('{:20}\t{:20}\t{}\t{}\t{}\t{}\n'.format('MeSH Label', 'Original', 'DUI', 'Start', 'End', 'MeSH Group'))
for label, code, op, start, end in zip(mti_output_dict['mesh_labels'][0],
                                       mti_output_dict['mesh_duis'][0],
                                       mti_output_dict['original_phrases'][0],
                                       mti_output_dict['mesh_term_token_start_idx'][0],
                                       mti_output_dict['mesh_term_token_end_idx'][0]):
    mesh_group = mesh_codes[code]['tree_DescriptorStringProcessed_1']
    print('{:20}\t{:20}\t{}\t{}\t{}\t{}'.format(label, op, code, start, end, mesh_group))

MeSH Label          	Original            	DUI	Start	End	MeSH Group

United Kingdom      	the UK              	D006113	65	71	geographic locations
Physics             	physics             	D010825	230	237	natural science disciplines
Physics             	Physics             	D010825	510	517	natural science disciplines
Neutrons            	neutron             	D009502	932	939	physical phenomena
Diamond             	Diamond             	D018130	1001	1008	inorganic chemicals
Protons             	protons             	D011522	1174	1181	physical phenomena
Ions                	ions                	D007477	1192	1196	inorganic chemicals
Neoplasms           	cancer              	D009369	1206	1212	None
Cell Proliferation  	proliferation       	D049109	1300	1313	physiological phenomena
Nuclear Energy      	nuclear energy      	D009678	1333	1347	environment and public health
Radioactive Waste   	radioactive waste   	D011850	1392	1409	environment and public health
Students            	graduate students

The terms above are printed from those labelled in the very first document in the database (shown in full above). The groups chosen are the 1st level, so not the most high level, and they seem to separate the terms into mostly sensible categories. There are mistakes due to the parsing method we have used for the MeSH XML file, however conersation is ongoing with NLM about how best to fix this.

## Sub-Topic Modelling

To demonstrate a potential use of the MeSH labels, we will have a quick look at topic modelling.

One issue with topic modelling on the entire dataset, is that only very coarse themes that are common across many documents are picked up. These often include very common health issues, or administrative jargon. Using the MeSH labels, we can create a subset of documents, and perform topic modelling on them instead, in a bid to find the topics that exist within any particular concept. We can imagine this as potentially useful as an innovation discovery tool for researchers wanting to find out more about a particular health domain.

As an example, we can use the MeSH label "Climate" to generate our data subset.

In [63]:
key_term = "'Climate'"

In [64]:
gdb_key_term_df = gdb_df[gdb_df['mesh_labels'].str.contains(key_term)]

In [295]:
len(gdb_key_term_df)

1693

This data subset consists of about 1700 documents.

First we parse the text, remove stop words and other non-useful tokens, and create ngrams.

In [ ]:
import spacy

nlp_sm = spacy.load('en_core_web_sm')

In [97]:
with open('../data/external/en_ranknl_long.txt', 'r') as f:
    text = f.read()
stop_words = text.split('\n')

In [98]:
for stop in stop_words:
    nlp_sm.vocab[stop].is_stop = True
    nlp_sm.vocab[stop.title()].is_stop = True
    nlp_sm.vocab[stop.upper()].is_stop = True

In [99]:
key_term_descriptions = [nlp_sm(d) for d in gdb_key_term_df['Description']]

In [161]:
key_term_description_processed_lemmas = []
for ktd in key_term_descriptions:
    lemmas = []
    for t in ktd:
        if t.is_stop:
            continue
        if t.is_punct:
            continue
        if t.is_bracket:
            continue
        if t.is_quote:
            continue
        if t.like_num:
            continue
        if t.is_digit:
            continue
        if t.like_url:
            continue
        if t.like_email:
            continue
        if t.is_space:
            continue
        if t.is_stop:
            continue
        if t.text.title() == key_term:
            continue
            #         if t.lower_ in stop_words:
#             continue
        lemmas.append(t.lower_ + '/' + t.pos_)
    key_term_description_processed_lemmas.append(lemmas)

In [162]:
from nlp_tools import GensimNGrammer

In [163]:
ngrammer = GensimNGrammer(n=2, **{'min_count': 30, 'threshold': 10})

In [164]:
key_term_description_processed_lemma_trigrams = ngrammer.fit_transform(key_term_description_processed_lemmas)

Looking at the most common term frequencies, we can see that even these contain fairly contextual information.

In [169]:
counts = Counter(flatten(key_term_description_processed_lemma_trigrams))

In [170]:
counts.most_common(20)

[('project/NOUN', 2589),
 ('climate/NOUN', 2199),
 ('data/NOUN', 2135),
 ('health/NOUN', 1385),
 ('provide/VERB', 1207),
 ('environmental/ADJ', 1182),
 ('development/NOUN', 1137),
 ('based/VERB', 1041),
 ('science/NOUN', 1027),
 ('models/NOUN', 1004),
 ('water/NOUN', 980),
 ('systems/NOUN', 950),
 ('impact/NOUN', 899),
 ('community/NOUN', 891),
 ('including/VERB', 878),
 ('energy/NOUN', 847),
 ('impacts/NOUN', 840),
 ('develop/VERB', 833),
 ('social/ADJ', 809),
 ('economic/ADJ', 797)]

Next we build the topic model. As we don't have a large number of documents, the number of topics is kept fairly low.

In [173]:
from gensim.models import LdaModel, CoherenceModel
from gensim.corpora import Dictionary

In [165]:
dictionary = Dictionary(key_term_description_processed_lemma_trigrams)

In [339]:
doc_bow = [dictionary.doc2bow(kt) for kt in key_term_description_processed_lemma_trigrams]

In [340]:
lda = LdaModel(corpus=doc_bow, id2word=dictionary, num_topics=100, chunksize=200, passes=10, minimum_probability=0.001, random_state=42)

/Users/grichardson/miniconda3/envs/rwjf/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [341]:
import pyLDAvis.gensim

In [342]:
lda_vis = pyLDAvis.gensim.prepare(lda, doc_bow, dictionary)
pyLDAvis.save_html(lda_vis, fig_path + '/gdb_climate_{date}_lda_viz.html'.format(date=today_str))

/Users/grichardson/miniconda3/envs/rwjf/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


The topics seem to be fairly representative of groups of intuitively connected concepts.

In [343]:
lda.print_topics(10)

[(3,
  '0.078*"disease/NOUN" + 0.072*"response/NOUN" + 0.041*"agent/NOUN" + 0.039*"infection/NOUN" + 0.026*"infectious/ADJ" + 0.025*"evidence/NOUN" + 0.024*"infected/VERB" + 0.024*"immune/ADJ" + 0.023*"virus/NOUN" + 0.022*"driven/VERB"'),
 (31,
  '0.046*"workshop/NOUN" + 0.041*"main/ADJ" + 0.030*"development/NOUN" + 0.026*"integration/NOUN" + 0.025*"market/NOUN" + 0.024*"conference/NOUN" + 0.023*"instrument/NOUN" + 0.021*"operations/NOUN" + 0.019*"offers/VERB" + 0.019*"work/NOUN"'),
 (95,
  '0.027*"proposed/VERB" + 0.024*"methods/NOUN" + 0.020*"performance/NOUN" + 0.016*"safety/NOUN" + 0.016*"large/ADJ" + 0.016*"quality/NOUN" + 0.015*"mechanisms/NOUN" + 0.015*"involves/VERB" + 0.014*"point/NOUN" + 0.013*"data/NOUN"'),
 (45,
  '0.036*"data/NOUN" + 0.021*"impacts/NOUN" + 0.017*"environmental/ADJ" + 0.015*"project/NOUN" + 0.013*"system/NOUN" + 0.012*"development/NOUN" + 0.010*"impact/NOUN" + 0.009*"climate/NOUN" + 0.009*"high/ADJ" + 0.009*"develop/VERB"'),
 (60,
  '0.046*"conditions/NOUN"

In [344]:
import gensim

In [345]:
lda_vecs = []
for db in doc_bow:
    lda_vecs.append(gensim.matutils.sparse2full(lda[db], length=100))

lda_vecs = np.array(lda_vecs)

In [346]:
from sklearn.manifold import TSNE

In [347]:
tsne = TSNE(random_state=42)

In [348]:
lda_tsne = tsne.fit_transform(lda_vecs)

In [351]:
lda_tsne_df = pd.DataFrame(lda_tsne)
lda_tsne_df.rename(columns={0: 'tsne_0', 1: 'tsne_1'}, inplace=True)
labels = [', '.join(list(set(ast.literal_eval(f)))) for f in gdb_key_term_df['mesh_labels'].values]
lda_tsne_df['labels'] = labels
lda_tsne_df['doc_id'] = gdb_key_term_df.index.values
lda_tsne_cds = ColumnDataSource(lda_tsne_df)

In [488]:
def clean_plot(p):
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

    p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
    p.yaxis.major_label_text_font_size = '0pt'  # turn off y-axis tick labels
    return p

In [492]:
hover = HoverTool(tooltips=[
    ("ID", "@doc_id"),
    ("Labels", "@labels"),
])

p = figure(width=800, height=600,
          x_axis_label='tsne_0', y_axis_label='tsne_1',
           title='Documents containing the MeSH label "Climate" transformed by LDA and t-SNE')
p.circle(x='tsne_0', y='tsne_1', source= lda_tsne_cds, size=7, alpha=0.5)
p.add_tools(hover)
p = clean_plot(p)

show(p)

Choosing one of the tight clusters at random, we can get a handful of document IDs, and fetch their details from the original GDB. 

In [353]:
gdb_df.iloc[[9035, 13419, 21148, 11818, 15795]][['Source ID', 'Description', 'country', 'Start Date']]

,Source ID,Description,country,Start Date
9035,GDB,Ecosystems and their biodiversity deliver prod...,United Kingdom,2012-04-01 00:00:00
13419,GDB,Ecosystems and their biodiversity deliver prod...,United Kingdom,2012-04-01 00:00:00
21148,GDB,Ecosystems and their biodiversity deliver prod...,United Kingdom,2012-04-01 00:00:00
11818,GDB,Ecosystems and their biodiversity deliver prod...,United Kingdom,2012-04-01 00:00:00
15795,GDB,Ecosystems and their biodiversity deliver prod...,United Kingdom,2012-04-01 00:00:00


This looks like the topics and the clusterings that we have may be due to duplicate entries in the dataset. This is possibly dominating the topic models, and also the TSNE model. 

Let's try to do this again on a de-duplicated "Climate" data subset. 

In [354]:
gdb_key_term_dedupe_df = gdb_key_term_df.drop_duplicates(subset='Description')

In [355]:
len(gdb_key_term_dedupe_df)

899

In [356]:
key_term_dedupe_descriptions = [nlp_sm(d) for d in gdb_key_term_dedupe_df['Description']]

In [357]:
key_term_dedupe_description_processed_lemmas = []
for ktd in key_term_dedupe_descriptions:
    lemmas = []
    for t in ktd:
        if t.is_stop:
            continue
        if t.is_punct:
            continue
        if t.is_bracket:
            continue
        if t.is_quote:
            continue
        if t.like_num:
            continue
        if t.is_digit:
            continue
        if t.like_url:
            continue
        if t.like_email:
            continue
        if t.is_space:
            continue
        if t.is_stop:
            continue
        if t.text.title() == key_term:
            continue
            #         if t.lower_ in stop_words:
#             continue
        lemmas.append(t.lower_ + '/' + t.pos_)
    key_term_dedupe_description_processed_lemmas.append(lemmas)

In [358]:
ngrammer = GensimNGrammer(n=2, **{'min_count': 15, 'threshold': 5})
key_term_description_dedupe_processed_lemma_trigrams = ngrammer.fit_transform(key_term_dedupe_description_processed_lemmas)

Looking at the most common term frequencies again, we can see a similar distribution, which is reassuring.

In [359]:
counts = Counter(flatten(key_term_description_dedupe_processed_lemma_trigrams))

In [ ]:
def print_term_counter(counter, n, low_pass=None):
    

In [360]:
counts.most_common(20)

[('project/NOUN', 1350),
 ('climate/NOUN', 1019),
 ('data/NOUN', 1003),
 ('environmental/ADJ', 566),
 ('health/NOUN', 532),
 ('water/NOUN', 526),
 ('students/NOUN', 524),
 ('development/NOUN', 522),
 ('systems/NOUN', 514),
 ('science/NOUN', 513),
 ('provide/VERB', 506),
 ('models/NOUN', 499),
 ('based/VERB', 454),
 ('including/VERB', 449),
 ('study/NOUN', 408),
 ('community/NOUN', 402),
 ('develop/VERB', 402),
 ('impacts/NOUN', 387),
 ('energy/NOUN', 381),
 ('impact/NOUN', 378)]

Next we build the topic model. As we don't have a large number of documents, the number of topics is kept fairly low. However, we should first remove some of the most commonly occuring words, as they will skew the topics significantly.

In [509]:
key_term_description_dedupe_processed_lemma_trigrams_low = [[t for t in terms if counts[t] < 1000] for terms in key_term_description_dedupe_processed_lemma_trigrams]

In [510]:
dictionary_dedupe = Dictionary(key_term_description_dedupe_processed_lemma_trigrams_low)

In [511]:
doc_bow_dedupe = [dictionary_dedupe.doc2bow(kt) for kt in key_term_description_dedupe_processed_lemma_trigrams_low]

In [513]:
lda_dedupe = LdaModel(corpus=doc_bow_dedupe, id2word=dictionary_dedupe, num_topics=50, chunksize=1000, passes=10, minimum_probability=0.001, random_state=42)

The topics seem to be fairly representative of groups of intuitively connected concepts.

In [514]:
lda_dedupe.print_topics(10)

[(14,
  '0.009*"health/NOUN" + 0.006*"students/NOUN" + 0.005*"science/NOUN" + 0.004*"resources/NOUN" + 0.004*"scientists/NOUN" + 0.004*"ocean/NOUN" + 0.004*"communities/NOUN" + 0.004*"infrastructure/NOUN" + 0.004*"environment/NOUN" + 0.004*"blue/ADJ"'),
 (47,
  '0.006*"food/NOUN" + 0.005*"factors/NOUN" + 0.005*"environmental/ADJ" + 0.004*"plant/NOUN" + 0.004*"soil/NOUN" + 0.004*"triassic/PROPN" + 0.004*"study/NOUN" + 0.004*"resilience/NOUN" + 0.004*"influenza/NOUN" + 0.003*"human/ADJ"'),
 (43,
  '0.006*"species/NOUN" + 0.006*"nitrogen/NOUN" + 0.005*"school/NOUN" + 0.005*"environmental/ADJ" + 0.004*"emissions/NOUN" + 0.004*"fellow/PROPN" + 0.004*"compounds/NOUN" + 0.004*"white/ADJ" + 0.003*"methods/NOUN" + 0.003*"populations/NOUN"'),
 (22,
  '0.008*"virus/NOUN" + 0.007*"proteins/NOUN" + 0.007*"disease/NOUN" + 0.005*"animals/NOUN" + 0.005*"impact/NOUN" + 0.004*"pollution/NOUN" + 0.004*"surface/NOUN" + 0.004*"context/NOUN" + 0.004*"time/NOUN" + 0.004*"immune/ADJ_system/NOUN"'),
 (42,
  '0

In [515]:
lda_vecs_dedupe = []
for db in doc_bow_dedupe:
    lda_vecs_dedupe.append(gensim.matutils.sparse2full(lda[db], length=100))

lda_vecs_dedupe = np.array(lda_vecs)

In [516]:
tsne_dedupe = TSNE(
            random_state=42,
            perplexity=30,
)
lda_tsne_dedupe = tsne.fit_transform(lda_vecs_dedupe)

In [517]:
lda_tsne_dedupe_df = pd.DataFrame(lda_tsne_dedupe)
lda_tsne_dedupe_df.rename(columns={0: 'tsne_0', 1: 'tsne_1'}, inplace=True)
labels_dedupe = [', '.join(list(set(ast.literal_eval(f)))) for f in gdb_key_term_dedupe_df['mesh_labels'].values]
lda_tsne_dedupe_df['labels'] = labels
lda_tsne_dedupe_df['doc_id'] = gdb_key_term_dedupe_df.index.values
lda_tsne_dedupe_cds = ColumnDataSource(lda_tsne_dedupe_df)

In [518]:
hover = HoverTool(tooltips=[
    ("ID", "@doc_id"),
    ("Labels", "@labels"),
])

p = figure(width=800, height=600,
           x_axis_label='tsne_0', y_axis_label='tsne_1',
           title='De-duplicated documents containing the MeSH label "Climate" transformed by LDA and t-SNE')
p.circle(x='tsne_0', y='tsne_1', source=lda_tsne_dedupe_cds, size=7, alpha=0.5)
p.add_tools(hover)
p = clean_plot(p)

show(p)

In [521]:
lda_vis = pyLDAvis.gensim.prepare(lda_dedupe, doc_bow_dedupe, dictionary_dedupe)
pyLDAvis.save_html(lda_vis, fig_path + '/gdb_climate_dedupe_{date}_lda_viz.html'.format(date=today_str))

/Users/grichardson/miniconda3/envs/rwjf/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


Now we can see that the very dense clusters have disappeared.

From topic modelling within an individual topic, we can see that it is possible to uncover the more nuanced topics within the documents covered by a broad term such as "Climate". This could be useful for helping users to discover new areas to focus on.